In [14]:
from qiskit import *
from qiskit.visualization import *
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.providers.ibmq import least_busy
from qiskit_aqua.algorithms import ExactEigensolver
from qiskit_aqua.algorithms import IQPE
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.initial_states import Custom
import numpy as np
import math

ModuleNotFoundError: No module named 'qiskit_aqua'

In [20]:
import qiskit
dir(qiskit)
qiskit.__qiskit_version__
%pip install -U qiskit

    100% |████████████████████████████████| 21.3MB 1.8MB/s eta 0:00:01
    100% |████████████████████████████████| 409kB 15.8MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/41/35/6b10aa3ea2fbe1b64cb0c3b2eaecb23cff044b574b151e8adc9ab9fbfe9a/qiskit_chemistry-0.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/01/29/b6ce038a75409239db8762b4744c8b53eb62e0b6240ece1c2969a7250206/pyeda-0.28.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/d8/23/c0add3524a58d13dbb6c3fa5f018d32104ef362788c4555a04c671948411/pyscf-1.6.1.post1-cp37-cp37m-manylinux1_x86_64.whl


  Complete output from command /home/archie/bin/anaconda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-3ictqpgi/qiskit/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-d2a_27f1 --python-tag cp37:
  running bdist_wheel
  
  ----------------------------------------
  Failed building wheel for qiskit
  Running setup.py clean for qiskit
  Complete output from command /home/archie/bin/anaconda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-3ictqpgi/pyeda/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-__ujha73 --python-tag cp37:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/pyeda
  copying pyeda/__in

Failed to build qiskit pyeda
  Found existing installation: qiskit-aer 0.2.0
    Uninstalling qiskit-aer-0.2.0:
      Successfully uninstalled qiskit-aer-0.2.0
  Running setup.py install for pyeda ... error
    Complete output from command /home/archie/bin/anaconda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-3ictqpgi/pyeda/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-mtdn_yoi/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.7
    creating build/lib.linux-x86_64-3.7/pyeda
    copying pyeda/__init__.py -> build/lib.linux-x86_64-3.7/pyeda
    copying pyeda/inter.py -> build/lib.linux-x86_64-3.7/pyeda
    copying pyeda/util.py -> build/lib.linux-x86_64-3.7/pyeda
    creating build/lib.linux-x86_64-3.7/pyed

Command "/home/archie/bin/anaconda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-3ictqpgi/pyeda/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-mtdn_yoi/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-3ictqpgi/pyeda/
Note: you may need to restart the kernel to use updated packages.


In [2]:
IBMQ.load_accounts()

In [4]:
def matrixHamiltonian(tam, freq): #tam 4, estados 0 a 3
    h = 6.582119624 * 10**(-16)
    E = [h*freq*(i+1/2) for i in range(tam)]
    L = []
    for i in range(tam):
        L.append(list())
        for j in range(tam):
            if i==j: L[i].append(E[i])
            else: L[i].append(0)
    m = np.matrix(L)
    return m


In [10]:
m = matrixHamiltonian(4, 10**16)
op = Operator(m).is_unitary()
eig_res = ExactEigensolver(op, k=1).run()
eigv = eig_res['eigvecs']

state = Custom(2, state_vector=eigv[0])

NameError: name 'ExactEigensolver' is not defined